# NVIDIA 데이터 큐레이션 종합 실습

이 노트북은 NeMo Curator를 사용한 데이터 큐레이션의 전체 워크플로우를 실습합니다.

## 목차
1. [환경 설정](#1.-환경-설정)
2. [데이터 품질 필터링](#2.-데이터-품질-필터링)
3. [중복 제거](#3.-중복-제거)
4. [PII 제거](#4.-PII-제거)
5. [독성 콘텐츠 필터링](#5.-독성-콘텐츠-필터링)
6. [종합 분석](#6.-종합-분석)

## 1. 환경 설정

### 1.1 필요한 라이브러리 Import

In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# NeMo Curator
from nemo_curator import ScoreFilter
from nemo_curator.datasets import DocumentDataset

print('환경 설정 완료')